# DATA STRUCTURES NECESSARY FOR THE SOLUTION OF NETWORK EQUATIONS  
Structures $S$: sparse matrices

## Matrices from network topology

- $A$: node-arcs incidence matrix of the graph: $M (m, n)$
- $A_r$: submatrix of A corresponding to the reservoirs: $M (m_r, n)$
- $A_d$: complementary submatrix of $Ar$ for $A$: $M (m_d, n)$
- $A_{dT}$: largest invertible square sub-matrix of $A_d$: $M (md, md)$
- $A_{dI}$: inverse matrix of $A_{dT}$: $M (m_d, m_d)$
- $A_{dC}$: complementary submatrix of $A_{dT}$ for $A_d$: $M (m_d, n - m_d)$
- $B$: arcs-cycle incidence matrix of the graph: $M (n, n-m_d)$

## Debit admissible

- $q_0$: vector of the admissible flow rates of the arcs: $M (n, 1)$


## Code

### import the modular

In [2]:
from numpy import dot
from numpy import zeros
from numpy import ones
from numpy import array
from numpy import int64
from numpy import concatenate
from numpy import random

from scipy import sparse
from scipy.sparse import eye
from scipy.sparse import coo_matrix
from scipy.sparse import vstack
from scipy.sparse.linalg import inv

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


### Problem $P\ $: parameterizable size problem

In [12]:
from Probleme_P import *

### Incidence matrix and associated sub-matrices

- Incidence matrix node-arcs of the graph

In [13]:
row_ind = concatenate((orig.astype(int64), dest.astype(int64)), axis=0)
col_ind = concatenate((array(range(0,n)), array(range(0,n))), axis=0)
mat_dat = concatenate((-ones(n), ones(n)), axis=0)
A = coo_matrix((mat_dat, (row_ind, col_ind)))

- Switch to integer CSC mode  
  $ A \cdot q=\left(\begin{array}{c}A_{r} \\ A_{d}\end{array}\right) \cdot q=\left(\begin{array}{c}f_{r} \\ f_{d}\end{array}\right) $

In [14]:
A = A.tocsc()
A = A.astype(int64)

- Partition of A according to the type of nodes  
  $ A_r q = f_n$

In [15]:
Ar = A[:mr,:]
Ad = A[mr:m,:]

- Sub-matrix of Ad associated with a tree and inverse  
  $ A_d q = f_d$

In [16]:
AdT = Ad[:,:md]
AdI = inv(AdT).astype(int64)

- Sub-matrix of Ad associated with a co-tree  
  $ A_{d}=\left(A_{d T}, A_{d C}\right) $  
  with  
  $ q=\left(\begin{array}{l}q_{T} \\ q_{C}\end{array}\right) $

In [17]:
AdC = Ad [:, md: n] 

- Matrix of incidence arcs-cycles  
  $ B=\left(\begin{array}{c}-\left(A_{d T}\right)^{-1} \cdot A_{d C} \\ I\end{array}\right)=\left(\begin{array}{c}B_{1} \\ B_{2}\end{array}\right) $

In [18]:
B1 = - dot(AdI, AdC)
B2 = eye(n-md, dtype=int64, format="csc")
B = vstack([B1,B2], format="csc")

### Vector of admissible flows   
  $$ q_{T}=q_{0}+B q_{C} $$  
  $$ q_{0}=\left(\begin{array}{c}\left(A_{d T}\right)^{-1} \cdot f_{d} \\ 0\end{array}\right) $$

In [19]:
q0 = zeros (n) 
q0 [: md ] = AdI.dot (fd)

In [3]:
qc = 0.1 * random.normal(size=n-md)

NameError: name 'n' is not defined